In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
%matplotlib widget

In [39]:
import pandas as pd
import os
from pathlib import Path
import wandb

os.chdir('/home/bohdan/studies/sem8/io')


In [40]:
csv_path = Path('medium.csv')
problem = pd.read_csv(csv_path)
problem.head()


,cust,x,y,demand,from,to,service
0,1,40.0,50.0,0.0,0.0,1236.0,0.0
1,2,45.0,68.0,10.0,912.0,967.0,90.0
2,3,45.0,70.0,30.0,825.0,870.0,90.0
3,4,42.0,66.0,10.0,65.0,146.0,90.0
4,5,42.0,68.0,10.0,727.0,782.0,90.0


1. Uruchomić
 - dostosowanie do naszych danych
 
2. Dodać modyfikacje
- dodajemy do mrówki wiedzę o depocie oraz o pojemności
- mrówka po osiągnięciu maks pojemności wraca do początku



In [56]:
from swarmlib_.aco4tsp.aco_algorithm import ACOAlgorithm
from swarmlib_.aco4tsp.vehicle import Vehicle


vehicles = [Vehicle(50, 1) for _ in range(10)]
config = {
    'ant_number': 1,
    'num_vehicles': 10,
    'iteration_number': 100,
    'commit_freq': 10,
    'vehicles': vehicles,
    'instance_path': csv_path
}

wandb.init(
    project="dynamic aco",
    config=config
)


# Logic goes here
try:
    solver = ACOAlgorithm(**config)
    best_plan, _ = solver.solve()
finally:
    wandb.finish()

# TODO

[ ] Poprawić wykresy funkcji kosztu:

* Powinien zawierać dwie wartości: 
    - aktualny koszt
    - najlepszy do tej pory znaleziony koszt

* Powinien mieć podpisane osie 



[ ] Zrozumienie (razem) i zaimplementowanie trzech metryk diversity tablice feromonów o których Byrski mówi w swoim artykule https://journals.agh.edu.pl/csci/article/view/4301


[ ] Przeszkuanie artykułów dot. mrówek do dynamicznego VRP


----------------------------------
- Jak pokazać, że coś się adaptuje?
- Funkcja kosztu jest czymś innym niż funkcja adaptacji 



-----

[ ] Dodajemy commitment (każdy pojazd będzie miał swoją prędkość per iterację)

[ ] Trasa do przejechania (po komitmencie) jest funkcją kosztu






